# Data Preprocessing/Cleaning

Before I start doing this notebook I will define what a delayed flight will be as this will be the principle used to drop or not some of the columns/features that come with this dataset, or to engineer others.

* First of all, a delay flight will be a flight that arrives late at its destination
* If the flight has any delays from its departure, but still arrives to its destination on time, it will not be considered a delayed flight

Furthermore, I will also handle this as a binary classification problem, with a 0 for a arrival on time, and a 1 for delayed arrival.

### Glossary
FL_DATE = Date of the Flight  
OP_CARRIER = Airline Identifier  
OP_CARRIER_FL_NUM = Flight Number  
ORIGIN = Starting Airport Code  
DEST = Destination Airport Code  
CRS_DEP_TIME = Planned Departure Time  
DEP_TIME = Actual Departure Time  
DEP_DELAY = Total Delay on Departure in minutes  
TAXI_OUT = The time duration elapsed between departure from the origin airport gate and wheels off  
WHEELS_OFF = The time point that the aircraft's wheels leave the ground  
WHEELS_ON = The time point that the aircraft'ss wheels touch on the ground  
TAXI_IN = The time duration elapsed between wheels-on and gate arrival at the destination airport  
CRS_ARR_TIME = Planned arrival time  
ARR_TIME = Actual Arrival Time = ARRIVAL_TIME - SCHEDULED_ARRIVAL  
ARR_DELAY = Total Delay on Arrival in minutes  
CANCELLED = Flight Cancelled (1 = cancelled)  
CANCELLATION_CODE = Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security  
DIVERTED = Aircraft landed on different airport that the one scheduled  
CRS_ELAPSED_TIME = Planned time amount needed for the flight trip  
ACTUAL_ELAPSED_TIME = AIR_TIME+TAXI_IN+TAXI_OUT  
AIR_TIME = The time duration between wheels_off and wheels_on time  
DISTANCE = Distance between two airports  
CARRIER_DELAY = Delay caused by the airline in minutes  
WEATHER_DELAY = Delay caused by weather  
NAS_DELAY = Delay caused by air system  
SECURITY_DELAY = caused by security reasons  
LATE_AIRCRAFT_DELAY = Delay caused by security  

## Libraries and Data Loading

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import datetime, scipy , warnings
warnings.filterwarnings('ignore')

In [2]:
df1 = pd.read_csv(r'C:\Users\USER\Downloads\archive\2018.csv')

In [3]:
df1.shape

(7213446, 28)

In [4]:
df1.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,0.0,NaN,0.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,0.0,NaN,0.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,0.0,NaN,0.0,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,1756,1754.0,-2.0,0.0,NaN,0.0,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,922,936.0,14.0,0.0,NaN,0.0,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213446 entries, 0 to 7213445
Data columns (total 28 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   OP_CARRIER_FL_NUM    int64  
 3   ORIGIN               object 
 4   DEST                 object 
 5   CRS_DEP_TIME         int64  
 6   DEP_TIME             float64
 7   DEP_DELAY            float64
 8   TAXI_OUT             float64
 9   WHEELS_OFF           float64
 10  WHEELS_ON            float64
 11  TAXI_IN              float64
 12  CRS_ARR_TIME         int64  
 13  ARR_TIME             float64
 14  ARR_DELAY            float64
 15  CANCELLED            float64
 16  CANCELLATION_CODE    object 
 17  DIVERTED             float64
 18  CRS_ELAPSED_TIME     float64
 19  ACTUAL_ELAPSED_TIME  float64
 20  AIR_TIME             float64
 21  DISTANCE             float64
 22  CARRIER_DELAY        float64
 23  WEATHER_DELAY        float64
 24

In [6]:
# Lets name our airlines
df1['OP_CARRIER'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
}, inplace=True)

In [7]:
df1['CANCELLED'].value_counts()

0.0    7096862
1.0     116584
Name: CANCELLED, dtype: int64

This is a binary column where number 1 equals a canceled flight, and a 0 not canceled, therefore we can drop the flights that were canceled, equivalent to 116584. Because cancelled flights are not considered to be delayed

In [9]:
df1 = df1[df1['CANCELLED'] == 0]
df1.head(2)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,United Airlines,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,0.0,NaN,0.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,United Airlines,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,0.0,NaN,0.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df1.drop(['CANCELLED'],axis = 1, inplace= True)

In [11]:
df1['CANCELLATION_CODE'].value_counts()

Series([], Name: CANCELLATION_CODE, dtype: int64)

In [13]:
df1['Unnamed: 27'].value_counts()

Series([], Name: Unnamed: 27, dtype: int64)

In [15]:
#So we the 'CANCELLATION_REASON' provided is a empty column so we can drop it and also drop 'Unnamed: 27'
df1.drop(['CANCELLATION_CODE','Unnamed: 27'],axis = 1, inplace = True)

### ORIGIN AND DESTINATION AIRPORT
These two column have the IATA airports codes for the origin and destination. I will try to change it to a real city name so that I can do flight analysis per city as well as part of the EDA.

To retrieve the details for the IATA codes you mentioned from the OpenFlights Airport Database,
Lets start by loading the file and by exploring it:

In [30]:
print(df1['ORIGIN'].nunique())
print(df1['DEST'].nunique())

358
358


In [32]:
df1_ORIGIN = df1['ORIGIN'].unique().tolist()
df1_DEST = df1['DEST'].unique().tolist()

In [33]:
for i in df1_DEST:
    if i not in df1_ORIGIN:
        print(i)

In [34]:
import pandas as pd

# Read the airport data from the CSV file
airport_data = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat",
                           header=None,
                           names=["Airport ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude", "Altitude", "Timezone", "DST", "Tz database time zone", "Type", "Source"])

# Select the rows with the desired IATA codes

selected_airports = airport_data.loc[airport_data['IATA'].isin(df1_ORIGIN)]

# Select only the desired columns
selected_airports = selected_airports[['IATA','Name', 'City', 'Country', 'Latitude', 'Longitude']]

# Print the resulting table
airports = pd.DataFrame(selected_airports)
airports.reset_index(drop = True,inplace = True)

In [35]:
for code in df1_DEST:
    if code not in airports['IATA'].to_list():
        print(code)

IFP
EAR


In [36]:
airports.head()

,IATA,Name,City,Country,Latitude,Longitude
0,PPG,Pago Pago International Airport,Pago Pago,American Samoa,-14.331000,-170.710007
1,SPN,Saipan International Airport,Saipan,Northern Mariana Islands,15.119000,145.729004
2,GUM,Antonio B. Won Pat International Airport,Agana,Guam,13.483400,144.796005
3,STT,Cyril E. King Airport,St. Thomas,Virgin Islands,18.337299,-64.973396
4,STX,Henry E Rohlsen Airport,St. Croix Island,Virgin Islands,17.701900,-64.798599


In [43]:
data = [
    ['IFP', 'Laughlin/Bullhead Airport', 'Bullhead City', 'United States', 35.157398, -114.5590],
    ['EAR', 'Kearney Regional Airport - LRY', 'Kearney', 'United States', 40.727001, -99.0068],
]

temp = pd.DataFrame(data, columns=['IATA', 'Name', 'City', 'Country', 'Latitude', 'Longitude'])

temp

,IATA,Name,City,Country,Latitude,Longitude
0,IFP,Laughlin/Bullhead Airport,Bullhead City,United States,35.157398,-114.5590
1,EAR,Kearney Regional Airport - LRY,Kearney,United States,40.727001,-99.0068


In [45]:
airports = pd.concat([airports, temp], ignore_index=True)

In [46]:
airports.shape

(358, 6)

In [49]:
airports.City.nunique()

345

There are some cities were repeated. Repeated, means that there are more than 1 airport in 1 city, therefore the number of unique values for CITY should be less than 358,i.e.345 

Now, the next step is to create a dictionary so that I can used it to replace the IATA_CODEs for ORIGIN and DEST on the df1 dataframe:

In [52]:
airport_dict = pd.Series(airports.City.values, index=airports.IATA).to_dict()
print(type(airport_dict))

<class 'dict'>


In [53]:
df1['ORIGIN'].replace(airport_dict, inplace=True)
df1['DEST'].replace(airport_dict, inplace=True)

In [54]:
df1.ORIGIN.nunique()

345

## Diverted

As a reminder, a DIVERTED flight is one that the plane has landed in a different airport than the one scheduled. This is normally temporary and the airline has the responsibility of taking passengers to their final destination without any extra cost. Therefore this is considered a delayed flight.

By doing a value counts we will see what type of values this feature contains:

In [55]:
df1.DIVERTED.value_counts()

0.0    7079005
1.0      17857
Name: DIVERTED, dtype: int64

It is clear that this is a binary feature, with 0 and 1, but I'm not sure what neither value means yet so I will need to analyze the entire column. The number of 1(es) is a lot smaller than the 0(es) so probably that is the way to go.

I will create a separate dataframe with only the columns where DIVERTED == 1 and then try to understand it based on the flight delays.

In [57]:
df1[df1['DIVERTED']==1]['ARR_DELAY'].unique()

array([nan])

In [58]:
# So basically there is no information to consider it as if it was delay or not,lets drop the column
df1.drop(['DIVERTED'],inplace = True, axis = 1)

In [59]:
df1.shape

(7096862, 24)

## Delay Reasons
On this section I will deal with 5 columns at the same time that are related:  

* CARRIER_DELAY
* WEATHER_DELAY
* NAS_DELAY
* SECURITY_DELAY
* LATE_AIRCRAFT_DELAY  

These 5 are all reasons for the flights to be delayed. Before I make any decision related to them, let's see how much information is in them first:

In [60]:
print(df1.CARRIER_DELAY.isna().sum())
print(df1.WEATHER_DELAY.isna().sum())
print(df1.NAS_DELAY.isna().sum())
print(df1.SECURITY_DELAY.isna().sum())
print(df1.LATE_AIRCRAFT_DELAY.isna().sum())

5744152
5744152
5744152
5744152
5744152


In [61]:
print("Percentage of valid data:", 100 - (df1.CARRIER_DELAY.isna().sum()*100/len(df1)))
print("Percentage of missing values:", (df1.CARRIER_DELAY.isna().sum()*100/len(df1)))

Percentage of valid data: 19.060677803795542
Percentage of missing values: 80.93932219620446


81% of data is missing and there is only 19% data presentl, Unfortunately this much is not enough to come to conlusions so Iam dropping these columns

In [62]:
df1 = df1.drop(['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'], axis=1)
print('The current shape of df1 is:', df1.shape)
df1.head(3).append(df1.tail(3))

The current shape of df1 is: (7096862, 19)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,2429,Newark,Denver,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,2427,Las Vegas,San Francisco,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,99.0,83.0,65.0,414.0
2,2018-01-01,United Airlines,2426,Santa Ana,Denver,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,134.0,126.0,106.0,846.0
7213443,2018-12-31,American Airlines,1817,Charlotte,Memphis,2015,2010.0,-5.0,36.0,2046.0,2114.0,4.0,2107,2118.0,11.0,112.0,128.0,88.0,511.0
7213444,2018-12-31,American Airlines,1818,Charlotte,Raleigh-durham,1300,1323.0,23.0,11.0,1334.0,1400.0,4.0,1350,1404.0,14.0,50.0,41.0,26.0,130.0
7213445,2018-12-31,American Airlines,1818,Raleigh-durham,Charlotte,1435,1443.0,8.0,8.0,1451.0,1535.0,7.0,1546,1542.0,-4.0,71.0,59.0,44.0,130.0


## OP_CARRIER_FL_NUM
We do not need this column so also drop this one

In [63]:
df1.drop(['OP_CARRIER_FL_NUM'], axis=1,inplace = True)

## DEP_TIME AND ARR_TIME
we are already having dep_delay and arr_delay which are most informative columns, and these columns are derived from dep time and arr time so it is better to drop these columns

In [64]:
df1.drop(columns=['DEP_TIME', 'ARR_TIME'], inplace=True)

In [65]:
df1.shape

(7096862, 16)

In [66]:
df1.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,Newark,Denver,1517,-5.0,15.0,1527.0,1712.0,10.0,1745,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,Las Vegas,San Francisco,1115,-8.0,11.0,1118.0,1223.0,7.0,1254,-24.0,99.0,83.0,65.0,414.0
2,2018-01-01,United Airlines,Santa Ana,Denver,1335,-5.0,15.0,1345.0,1631.0,5.0,1649,-13.0,134.0,126.0,106.0,846.0
3,2018-01-01,United Airlines,Fort Myers,Chicago,1546,6.0,19.0,1611.0,1748.0,6.0,1756,-2.0,190.0,182.0,157.0,1120.0
4,2018-01-01,United Airlines,Chicago,Albany,630,20.0,13.0,703.0,926.0,10.0,922,14.0,112.0,106.0,83.0,723.0


In [67]:
df1.to_csv('df1.csv')

### Dealing with Missing Values - NaN


In [23]:
import pandas as pd
df1 = pd.read_csv('df1.csv',index_col = 0)

In [24]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7096862 entries, 0 to 7213445
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   ORIGIN               object 
 3   DEST                 object 
 4   CRS_DEP_TIME         int64  
 5   DEP_DELAY            float64
 6   TAXI_OUT             float64
 7   WHEELS_OFF           float64
 8   WHEELS_ON            float64
 9   TAXI_IN              float64
 10  CRS_ARR_TIME         int64  
 11  ARR_DELAY            float64
 12  CRS_ELAPSED_TIME     float64
 13  ACTUAL_ELAPSED_TIME  float64
 14  AIR_TIME             float64
 15  DISTANCE             float64
dtypes: float64(10), int64(2), object(4)
memory usage: 920.5+ MB


In [25]:
df1.isna().sum()

FL_DATE                    0
OP_CARRIER                 0
ORIGIN                     0
DEST                       0
CRS_DEP_TIME               0
DEP_DELAY               4743
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON               2662
TAXI_IN                 2662
CRS_ARR_TIME               0
ARR_DELAY              20456
CRS_ELAPSED_TIME           7
ACTUAL_ELAPSED_TIME    17858
AIR_TIME               17858
DISTANCE                   0
dtype: int64

In [26]:
df_NaN = df1.loc[df1.isnull().any(axis=1)]
print(df_NaN.shape)
df_NaN.head()

(25045, 16)


,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
1168,2018-01-01,United Airlines,San Francisco,San Diego,729,-11.0,17.0,735.0,1124.0,3.0,903,NaN,94.0,NaN,NaN,447.0
1638,2018-01-01,Alaska Airlines,Petersburg,Wrangell,1514,72.0,20.0,1646.0,NaN,NaN,1538,NaN,24.0,NaN,NaN,31.0
1643,2018-01-01,Alaska Airlines,Ketchikan,Wrangell,925,-7.0,26.0,944.0,NaN,NaN,1011,NaN,46.0,NaN,NaN,82.0
1683,2018-01-01,Alaska Airlines,Seattle,San Diego,550,-6.0,18.0,602.0,1046.0,3.0,840,NaN,170.0,NaN,NaN,1050.0
1850,2018-01-01,Alaska Airlines,Seattle,San Diego,725,-3.0,16.0,738.0,1216.0,3.0,1017,NaN,172.0,NaN,NaN,1050.0


There are a total of 25,045 rows with missing values out of 7,096,862 rows of data. This represents:



In [27]:
(len(df_NaN)/len(df1))*100

0.352902451816028

Percentage of rows with at least 1 NaN value: 0.352902451816028

In [28]:
df1.dropna(inplace = True)

In [29]:
print(df1.shape)
df1.isna().sum()

(7071817, 16)


FL_DATE                0
OP_CARRIER             0
ORIGIN                 0
DEST                   0
CRS_DEP_TIME           0
DEP_DELAY              0
TAXI_OUT               0
WHEELS_OFF             0
WHEELS_ON              0
TAXI_IN                0
CRS_ARR_TIME           0
ARR_DELAY              0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
dtype: int64

In [30]:
df1.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,Newark,Denver,1517,-5.0,15.0,1527.0,1712.0,10.0,1745,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,Las Vegas,San Francisco,1115,-8.0,11.0,1118.0,1223.0,7.0,1254,-24.0,99.0,83.0,65.0,414.0
2,2018-01-01,United Airlines,Santa Ana,Denver,1335,-5.0,15.0,1345.0,1631.0,5.0,1649,-13.0,134.0,126.0,106.0,846.0
3,2018-01-01,United Airlines,Fort Myers,Chicago,1546,6.0,19.0,1611.0,1748.0,6.0,1756,-2.0,190.0,182.0,157.0,1120.0
4,2018-01-01,United Airlines,Chicago,Albany,630,20.0,13.0,703.0,926.0,10.0,922,14.0,112.0,106.0,83.0,723.0


Time-Related Columns  

Time is normally a categorical and having it in the current format will give us too many columns when the hot encode is applied to them, therefore I will split the time into 4 quarters of the days meaning of 6 hours each.

In [31]:
df1['CRS_DEP_TIME'] = np.ceil(df1['CRS_DEP_TIME']/600).apply(int)
df1['WHEELS_OFF'] = np.ceil(df1['WHEELS_OFF']/600).apply(int) 
df1['WHEELS_ON'] = np.ceil(df1['WHEELS_ON']/600).apply(int)
df1['CRS_ARR_TIME'] = np.ceil(df1['CRS_ARR_TIME']/600).apply(int)

example for wat happened above 'CRS_DEP_TIME' column from a format of HHMM (e.g. 0845) to an integer format of the hour of the day (e.g. 9).

In [32]:
df1.head(3)

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,Newark,Denver,3,-5.0,15.0,3,3,10.0,3,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,Las Vegas,San Francisco,2,-8.0,11.0,2,3,7.0,3,-24.0,99.0,83.0,65.0,414.0
2,2018-01-01,United Airlines,Santa Ana,Denver,3,-5.0,15.0,3,3,5.0,3,-13.0,134.0,126.0,106.0,846.0


## Feature Extraction

In [33]:
df1['DAY'] = pd.DatetimeIndex(df1['FL_DATE']).day
df1['MONTH'] = pd.DatetimeIndex(df1['FL_DATE']).month


In [34]:
import calendar
df1['MONTH_AB'] = df1['MONTH'].apply(lambda x: calendar.month_abbr[x])

### Binary Classification

As I mentioned at the beginning of this document, this is a binary classification, which means that I will run my models with the target being a column that I will engineer called FLIGHT_STATUS. In this column there will be only two values (hence the name binary), a 0 for flights that arrive either earlier or on time(+ 5min), and a 1 for flights that are delayed for more than 5 minutes.

In [52]:
pd.options.display.float_format = '{:.2f}'.format
df1['ARR_DELAY'].describe()

count   7071817.00
mean          5.06
std          46.94
min        -120.00
25%         -14.00
50%          -6.00
75%           8.00
max        2692.00
Name: ARR_DELAY, dtype: float64

In [42]:
df1['FLIGHT_STATUS'] = df1['ARR_DELAY'].apply(lambda x : 0 if x <= 5 else 1)

In [55]:
df1.FLIGHT_STATUS.value_counts(normalize=True)

0   0.72
1   0.28
Name: FLIGHT_STATUS, dtype: float64

In [56]:
df1.to_csv('df1.csv')

In [57]:
df1.isna().sum()


FL_DATE                0
OP_CARRIER             0
ORIGIN                 0
DEST                   0
CRS_DEP_TIME           0
DEP_DELAY              0
TAXI_OUT               0
WHEELS_OFF             0
WHEELS_ON              0
TAXI_IN                0
CRS_ARR_TIME           0
ARR_DELAY              0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
DAY                    0
MONTH                  0
MONTH_AB               0
FLIGHT_STATUS          0
dtype: int64